In [1]:
import recordlinkage
from recordlinkage.index import Block
import pandas as pd

## Basic example

In [2]:
# Hypothetical datasets
data1 = {
    'Name': ['David Smith', 'Bill Johnson', 'Charles Brown'],
    'DateOfBirth': ['1990-01-01', '1985-05-12', '1980-07-23'],
    'Gender': ['F', 'M', 'M'],
    'Address': ['123 Main St', '456 Elm St', '789 Oak St']
}

data2 = {
    'Name': ['David Smith', 'Robert Johnson', 'Chad Brade'],
    'DateOfBirth': ['1990-01-01', '1985-05-12', '1980-07-25'],
    'Gender': ['F', 'M', 'M'],
    'Address': ['123 Main Street', '456 Elm Street', '789 Oak Street']
}

df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)

In [3]:
df1

,Name,DateOfBirth,Gender,Address
0,David Smith,1990-01-01,F,123 Main St
1,Bill Johnson,1985-05-12,M,456 Elm St
2,Charles Brown,1980-07-23,M,789 Oak St


In [4]:
df2

,Name,DateOfBirth,Gender,Address
0,David Smith,1990-01-01,F,123 Main Street
1,Robert Johnson,1985-05-12,M,456 Elm Street
2,Chad Brade,1980-07-25,M,789 Oak Street


In [5]:
indexer = recordlinkage.Index()
indexer.full()
pairs = indexer.index(df1, df2)

In [6]:
indexer = recordlinkage.Index()
indexer.block("DateOfBirth")
candidate_links = indexer.index(df1, df2)
len(candidate_links)

2

In [7]:
# Initialize the comparison object
compare = recordlinkage.Compare()

In [8]:
# Add comparison rules
compare.string('Name', 'Name', method='jarowinkler', threshold=0.85, label='Name')
compare.exact('Gender', 'Gender', label='Gender')
compare.string('DateOfBirth', 'DateOfBirth', method='jarowinkler', threshold=0.85, label='DateOfBirth')
compare.string('Address', 'Address', method='jarowinkler', threshold=0.85, label='Address')
features = compare.compute(candidate_links, df1, df2)

In [9]:
# Identify matches
matches = features[features.sum(axis=1) > 3].reset_index()
matches = matches.set_index(['level_0', 'level_1'])

# Output match results
print('Matched Records:')
for index in matches.index:
    print(f'Data1 record: {index[0]} matches Data2 record: {index[1]}')

Matched Records:
Data1 record: 0 matches Data2 record: 0


In [10]:
matches

,,Name,Gender,DateOfBirth,Address
level_0,level_1,,,,
0,0,1.0,1,1.0,1.0


## Linkage detection and merging example

In [11]:
import pandas as pd
import recordlinkage

# Define the datasets as dataframes with additional columns in df1
data1 = {
    "id": [1,2,3],
    'Name': ['David Smith', 'Bill Johnson', 'Charles Brown'],
    'DateOfBirth': ['1990-01-01', '1985-05-12', '1980-07-23'],
    'Gender': ['F', 'M', 'M'],
    'Address': ['123 Main St', '456 Elm St', '789 Oak St'],
    'Phone': ['123-456-7890', '234-567-8901', '345-678-9012'],  # Additional column
    'Email': ['david@example.com', 'bill@example.com', 'charles@example.com']  # Additional column
}

data2 = {
    "id": [101,102,103],
    'Name': ['David Smith', 'Robert Johnson', 'Chad Brade'],
    'DateOfBirth': ['1990-01-01', '1985-05-12', '1980-07-25'],
    'Gender': ['F', 'M', 'M'],
    'Address': ['123 Main Street', '456 Elm Street', '789 Oak Street']
}

df1 = pd.DataFrame(data1).set_index("id")
df2 = pd.DataFrame(data2).set_index("id")

# Step 1: Create an indexer to link potential matches between the two datasets
indexer = recordlinkage.Index()
indexer.full()  # Or use .block() if there's a field for blocking, like 'Gender'

# Generate candidate pairs
candidate_links = indexer.index(df1, df2)

# Step 2: Define comparison criteria
compare = recordlinkage.Compare()
compare.string('Name', 'Name', method='jarowinkler', threshold=0.85)
compare.exact('DateOfBirth', 'DateOfBirth')
compare.string('Address', 'Address', method='jarowinkler', threshold=0.85)

# Compute the comparison vectors
comparison_vectors = compare.compute(candidate_links, df1, df2)

# Step 3: Classify matches based on a threshold
matches = comparison_vectors[comparison_vectors.sum(axis=1) > 2]

# Step 4: Convert matched pairs to a DataFrame and rename columns
matched_indices = matches.index.to_frame(index=False)
matched_indices.columns = ['df1_index', 'df2_index']

# Step 5: Merge matched pairs with df2 and only the additional columns from df1
additional_columns = ['Phone', 'Email']  # Define the additional columns
linked_df = matched_indices.merge(df2, left_on='df2_index', right_index=True)\
                           .merge(df1[additional_columns], left_on='df1_index', right_index=True)



In [12]:
linked_df

,df1_index,df2_index,Name,DateOfBirth,Gender,Address,Phone,Email
0,1,101,David Smith,1990-01-01,F,123 Main Street,123-456-7890,david@example.com


In [13]:
matches

,,0,1,2
id_1,id_2,,,
1,101,1.0,1,1.0
